In [16]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
model.invoke("안녕! 수요일 밤에 공부하러 왔어.")


AIMessage(content='안녕하세요! 수요일 밤에 공부하러 오셨군요. 어떤 과목이나 주제를 공부하고 계신가요? 도움이 필요하시면 말씀해 주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 21, 'total_tokens': 57, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CBhECYdnbdWyWcnNQN2sUkda7YWpN', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--06203d89-6a15-4ec1-abd6-ae189bb55f92-0', usage_metadata={'input_tokens': 21, 'output_tokens': 36, 'total_tokens': 57, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [17]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list[str], add_messages]

graph_builder = StateGraph(State)

In [18]:
from langchain_core.tools import tool
from datetime import datetime
import pytz
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

import bs4
from langchain_community.document_loaders import WebBaseLoader

@tool
def get_current_time(timezone: str = "Asia/Seoul", location: str = "서울") -> str:
    """
    현재 시간을 조회합니다.

    Args:
        timezone (str): 타임존
        location (str): 위치

    Returns:
        str: 현재 시간
    """
    try:
        tz = pytz.timezone(timezone)
        now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
        return f"{timezone} ({location}) 현재 시각 {now}"
    except pytz.UnknownTimeZoneError:
        return f"알 수 없는 타임존: {timezone}"

@tool
def get_search_web(query: str, search_period: str = "m") -> str:
    """
    웹 검색을 수행하는 함수.

    Args:
        query (str): 검색할 키워드
        search_period (str): 검색 기간 (h: 시간, d: 일, w: 주, m: 월, y: 년)

    Returns:
        str: 검색 결과
    """
    wrapper = DuckDuckGoSearchAPIWrapper(
        time = search_period
    )

    print("\n----- WEB SEARCH -----")
    print(f"# Query: {query}")
    print(f"# Search Period: {search_period}")

    search = DuckDuckGoSearchResults(
        api_wrapper=wrapper,
        results_separator=";\n"
    )

    searched = search.invoke(query)
    for i, result in enumerate(searched.split(";\n")):
        print(f"{i+1}. {result}")
    
    return searched
    
# 도구 바인딩
tools = [get_current_time, get_search_web]
    
    



In [19]:
print(tools[0].invoke({"timezone": "Asia/Seoul", "location": "서울"}))
print(tools[0].invoke({"timezone": "Europe/London", "location": "런던"}))

Asia/Seoul (서울) 현재 시각 2025-09-03 21:50:13
Europe/London (런던) 현재 시각 2025-09-03 13:50:13


In [20]:
tools[1].invoke({"query": "GTA 6", "search_period": "m"})


----- WEB SEARCH -----
# Query: GTA 6
# Search Period: m
1. snippet: When is GTA 6 coming out? Given how many years it takes to create open worlds of the same caliber as GTA 5, it's no surprise that Rockstar Games has ..., title: GTA 6 release date and time, trailers, and latest news, link: https://www.pcgamesn.com/grand-theft-auto-vi/gta-6-release-date-setting-map-characters-gameplay-trailers
2. snippet: The first GTA 6 announcement came in February 2022, when Rockstar Games snuck out the news that GTA 6 is "well underway" as part of a broader GTA 5 ..., title: GTA 6: Everything we know so far | GamesRadar+, link: https://www.gamesradar.com/gta-6-guide/
3. snippet: While we wait for GTA 6 , rumors suggest that GTA 4 is getting a remaster Grand Theft Auto 4 is reportedly coming to PS5, Xbox Series X, and Xbox ..., title: GTA 6 - everything we know so far | TechRadar, link: https://www.techradar.com/gaming/gta-6
4. snippet: The second, and latest, GTA 6 trailer released on 6th May 2025

'snippet: When is GTA 6 coming out? Given how many years it takes to create open worlds of the same caliber as GTA 5, it\'s no surprise that Rockstar Games has ..., title: GTA 6 release date and time, trailers, and latest news, link: https://www.pcgamesn.com/grand-theft-auto-vi/gta-6-release-date-setting-map-characters-gameplay-trailers;\nsnippet: The first GTA 6 announcement came in February 2022, when Rockstar Games snuck out the news that GTA 6 is "well underway" as part of a broader GTA 5 ..., title: GTA 6: Everything we know so far | GamesRadar+, link: https://www.gamesradar.com/gta-6-guide/;\nsnippet: While we wait for GTA 6 , rumors suggest that GTA 4 is getting a remaster Grand Theft Auto 4 is reportedly coming to PS5, Xbox Series X, and Xbox ..., title: GTA 6 - everything we know so far | TechRadar, link: https://www.techradar.com/gaming/gta-6;\nsnippet: The second, and latest, GTA 6 trailer released on 6th May 2025 , four days after Rockstar announced that the game was delate

In [21]:
for tool in tools:
    print(tool.name, tool)


get_current_time name='get_current_time' description='현재 시간을 조회합니다.\n\nArgs:\n    timezone (str): 타임존\n    location (str): 위치\n\nReturns:\n    str: 현재 시간' args_schema=<class 'langchain_core.utils.pydantic.get_current_time'> func=<function get_current_time at 0x119608ae0>
get_search_web name='get_search_web' description='웹 검색을 수행하는 함수.\n\nArgs:\n    query (str): 검색할 키워드\n    search_period (str): 검색 기간 (h: 시간, d: 일, w: 주, m: 월, y: 년)\n\nReturns:\n    str: 검색 결과' args_schema=<class 'langchain_core.utils.pydantic.get_search_web'> func=<function get_search_web at 0x11a7cb9c0>


In [ ]:
model_with_tools = model.bind_tools(tools)
